# Algoritmo Sistema de Recomendación

A continuación, se dará el código en JupyterNotebook para el algoritmo de sistema de recomendación.

In [ ]:
 # importar librerias
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("white")

- Este fragmento importa las librerías necesarias para el análisis de datos y la visualización. **`%matplotlib inline`** es un comando específico de Jupyter Notebook para mostrar gráficos dentro del mismo notebook.
- Luego se importan `pandas`, `numpy`, `matplotlib.pyplot` y `seaborn`.

In [ ]:
route = "C:/Users/ASUS/PythonNotebook/Datasets_CSV/"
colum_names = ["user_id", "item_id", "rating", "timestamp"]
dataframe = pd.read_csv(route + "u.data", sep = "\t", names = colum_names)
dataframe.head()

- Leemos un conjunto de datos desde un archivo CSV llamado "u.data" en una variable llamada **`dataframe`**.
- Separa las columnas por tabulaciones y asigna nombres a las columnas.

In [ ]:
movie_titles = pd.read_csv(route + "Movie_Id_Titles")
movie_titles.head()

Lee el archivo "Movie_Id_Titles" que contiene los títulos de las películas.

In [ ]:
dataframe = pd.merge(dataframe, movie_titles, on = "item_id")
dataframe.head()

Combina los datos de calificaciones con los títulos de películas basándose en el ID de la película.

In [ ]:
# clasificar (agrupar) por titulo y puntuacion
# contar calificaciones (.count())
dataframe.groupby("title")["rating"].count().sort_values(ascending = False).head()

In [ ]:
# media de calificaciones (.mean())
dataframe.groupby("title")["rating"].mean().sort_values(ascending = False).head()

Utiliza el método groupby para calcular el número de calificaciones y la media de calificaciones por título de película.

In [ ]:
# obtener media de rating por cada pelicula
ratings = pd.DataFrame(dataframe.groupby("title")["rating"].mean())
ratings.head()

In [ ]:
# añadir columna (numero de calificaciones) a ratings
ratings["num_of_ratings"] = pd.DataFrame(dataframe.groupby("title")["rating"].count())
ratings.head()

Crea un DataFrame llamado ratings que contiene la media de calificaciones y el número de calificaciones para cada película.

In [ ]:
# histograma de numero de ratings
plt.figure(figsize = (10, 4))
plt.title("Num of ratings histogram")
plt.xlabel("Num Movies")
plt.ylabel("Num Users")
ratings["num_of_ratings"].hist(bins = 70)

In [ ]:
# histograma de ratings
plt.figure(figsize = (10, 4))
plt.title("Rating histogram")
plt.xlabel("Rating")
plt.ylabel("Num Users")
ratings["rating"].hist(bins = 70)

Crea histogramas para visualizar la distribución del número de calificaciones y de las calificaciones.

In [ ]:
# grafico conjunto de ratings y numero de ratings
sns.jointplot(x = "rating", y = "num_of_ratings", data = ratings, alpha = 0.5)

Crea un gráfico conjunto utilizando sns.jointplot para visualizar la relación entre la calificación media y el número de calificaciones.

In [ ]:
# pre procesamiento de datos
# datos de calificacion para cada pelicula por cada usuario 
movieMat = dataframe.pivot_table(index = "user_id", columns = "title", values = "rating")
movieMat.head()

Utiliza pivot_table para crear una matriz de películas y usuarios donde las filas son usuarios, las columnas son películas y los valores son las calificaciones dadas por los usuarios.

In [ ]:
# listar peliculas con mejor media de rating y su numero de ratings
ratings.sort_values("num_of_ratings", ascending = False).head(10)

In [ ]:
# obtener valores de dos peliculas
starwars_user_ratings = movieMat["Star Wars (1977)"]
liarliar_user_ratings = movieMat["Liar Liar (1997)"]
starwars_user_ratings.head(10)

In [ ]:
# Correlación entre peliculas mas valoradas
similarToStarwars = movieMat.corrwith(starwars_user_ratings)
similarToLiarliar = movieMat.corrwith(liarliar_user_ratings)

In [ ]:
# el codigo anterior tira error porque hay valores nulos, ya que no todos los usuarios
# calificaron todas las peliculas
corrStarwars = pd.DataFrame(similarToStarwars, columns = ["Correlation"])
corrStarwars.dropna(inplace = True)
corrStarwars.head()

In [ ]:
# ordenar datos por correlacion
corrStarwars.sort_values("Correlation", ascending = False).head(10)

- Calcula la correlación entre las calificaciones de "Star Wars (1977)" y todas las demás películas, y entre las calificaciones de "Liar Liar (1997)" y todas las demás películas.
- Limpia los valores nulos y muestra las películas más correlacionadas con "Star Wars" y "Liar Liar", respectivamente.

In [ ]:
# filtrar peliculas
corrStarwars = corrStarwars.join(ratings["num_of_ratings"])
corrStarwars.head(10)

In [ ]:
# filtrar por numero de ratings mator a 100
corrStarwars[corrStarwars["num_of_ratings"] > 100].sort_values("Correlation", ascending = False).head(10)

Filtra las películas basándose en el número mínimo de calificaciones (mayor a 100) y muestra las películas más correlacionadas con "Star Wars" y "Liar Liar" que cumplen con este criterio.

In [ ]:
# repetir proceso para otra pelicula
corrLiarliar = pd.DataFrame(similarToLiarliar, columns = ["Correlation"])
corrLiarliar.dropna(inplace = True)
corrLiarliar = corrLiarliar.join(ratings["num_of_ratings"])
corrLiarliar[corrLiarliar["num_of_ratings"] > 100].sort_values("Correlation", ascending = False).head(10)

Se repite el mismo proceso para la otra pelicula “Liar Liar”.